In [2]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pickle

company_names = {}

def fetch_job_listings(page):
    url = f"https://www.dice.com/jobs?page={page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.find_all('div', class_='header -mb-2 flex flex-row justify-between')

def get_company_links(companies):
    for company in companies:
        name_company = company.find('p').text.strip()
        link = 'https://www.dice.com' + company.find('a').get('href')
        company_names[name_company] = link

In [64]:
for page in range(1, 26):
    companies = fetch_job_listings(page)
    print(f"Page {page} companies successfully fetched.")
    time.sleep(random.uniform(1, 3))
    get_company_links(companies)
print(f'Number of company links = {len(company_names)}')

Page 1 companies successfully fetched.
Page 2 companies successfully fetched.
Page 3 companies successfully fetched.
Page 4 companies successfully fetched.
Page 5 companies successfully fetched.
Page 6 companies successfully fetched.
Page 7 companies successfully fetched.
Page 8 companies successfully fetched.
Page 9 companies successfully fetched.
Page 10 companies successfully fetched.
Page 11 companies successfully fetched.
Page 12 companies successfully fetched.
Page 13 companies successfully fetched.
Page 14 companies successfully fetched.
Page 15 companies successfully fetched.
Page 16 companies successfully fetched.
Page 17 companies successfully fetched.
Page 18 companies successfully fetched.
Page 19 companies successfully fetched.
Page 20 companies successfully fetched.
Page 21 companies successfully fetched.
Page 22 companies successfully fetched.
Page 23 companies successfully fetched.
Page 24 companies successfully fetched.
Page 25 companies successfully fetched.
Number of

In [3]:
import pickle

with open('company_links.pkl', 'wb') as f:
  pickle.dump(company_names, f)
print("Saved: company_links.pkl")

Saved: company_links.pkl


In [5]:
with open('company_links.pkl', 'rb') as f:
  loaded_config = pickle.load(f)
print("Loaded company_links.pkl successfully.")
print(f"แสดงข้อมูลที่อ่านได้: {len(loaded_config)} บริษัท")
print(f"อ่านข้อมูลจาก company_links.pkl: {loaded_config}")
print(f"เช็ค Data Type: {type(loaded_config)}")

Loaded company_links.pkl successfully.
แสดงข้อมูลที่อ่านได้: 109 บริษัท
อ่านข้อมูลจาก company_links.pkl: {'Navy Federal Credit Union': 'https://www.dice.com//company-profile/5bea0a06-736c-43a2-8da0-ef013448b47e?companyname=Navy%20Federal%20Credit%20Union', 'Ducommun, Inc.': 'https://www.dice.com//company-profile/758386a8-82c4-5e14-887c-4b5f2a3011ff?companyname=Ducommun%2C%20Inc.', 'InfoPeople Corp': 'https://www.dice.com//company-profile/b19aac2b-eec1-55c3-bd54-0b0909fe37b9?companyname=InfoPeople%20Corp', 'Anblicks': 'https://www.dice.com//company-profile/d2b2b145-68b0-5804-97b1-deaf9ff4935d?companyname=Anblicks', 'SolveIT Services Inc': 'https://www.dice.com//company-profile/e2569599-ccbc-527a-863e-01f1e8ab7c80?companyname=SolveIT%20Services%20Inc', 'MSR Technology Group LLC': 'https://www.dice.com//company-profile/2e891845-d492-4e27-a059-ab4377071d7e?companyname=MSR%20Technology%20Group%20LLC', 'Infomerica, Inc': 'https://www.dice.com//company-profile/a6b09e9a-fecd-51da-a275-5d61d9e1

Selenium

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [103]:
def get_company_job_links(company_url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    all_job_links = []
    
    try:
        # เริ่มต้นที่หน้าแรกก่อนเพื่อเช็คจำนวนหน้าทั้งหมด
        driver.get(company_url)
        wait = WebDriverWait(driver, 15)
        
        # 1. เข้าไปที่ Jobs Tab เพื่อให้ Shadow DOM ของรายการงานโหลด
        jobs_tab = wait.until(EC.element_to_be_clickable((By.XPATH, "//li[contains(text(), 'Jobs')]")))
        jobs_tab.click()
        
        # รอให้รายการงานปรากฏ
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "dhi-job-search-job-list")))
        time.sleep(4) 

        def get_total_pages():
            script = """
            const pagination = document.querySelector('dhi-job-search-job-list')?.shadowRoot
                               ?.querySelector('seds-pagination')?.shadowRoot.querySelector('.page');
            if (!pagination) return 1;
            const spans = pagination.querySelectorAll('span');
            for (let s of spans) {
                if (s.innerText.includes('of')) {
                    return parseInt(s.innerText.replace(/[^0-9]/g, ''));
                }
            }
            return 1;
            """
            return int(driver.execute_script(script))

        total_pages = get_total_pages()
        #print(f"Detected Total Pages: {total_pages}")

        # สคริปต์ดึง Link (เจาะ Shadow DOM)
        pierce_links_script = """
        function findAllLinksDeep(selector, root = document, links = []) {
            const foundInRoot = root.querySelectorAll(selector);
            foundInRoot.forEach(el => { if (el.href) links.push(el.href); });
            const hosts = root.querySelectorAll('*');
            for (const host of hosts) {
                if (host.shadowRoot) findAllLinksDeep(selector, host.shadowRoot, links);
            }
            return links;
        }
        return findAllLinksDeep('a.job-title-link');
        """

        for page_num in range(1, total_pages + 1):
            # สร้าง URL สำหรับแต่ละหน้า
            # ตรวจสอบว่ามี ? อยู่ใน URL เดิมหรือยังเพื่อเลือกใช้ & หรือ ?
            separator = "&" if "?" in company_url else "?"
            page_url = f"{company_url}{separator}page={page_num}"
            
            #print(f"Scraping page {page_num}: {page_url}")
            driver.get(page_url)
            
            # หลังจากเปลี่ยน URL ต้องรอให้ Shadow DOM โหลดใหม่
            time.sleep(5) 
            
            # ดึงข้อมูล Link
            current_links = driver.execute_script(pierce_links_script)
            
            initial_count = len(all_job_links)
            for link in current_links:
                if link not in all_job_links:
                    all_job_links.append(link)
            
            #print(f"Added {len(all_job_links) - initial_count} new links.")

        #print(f"Extraction complete. Found {len(all_job_links)} total links.")
        
    finally:
        driver.quit()
    return all_job_links

In [104]:
all_jobs = []
for company_url in company_names.values():
    jobs_link = get_company_job_links(company_url)
    all_jobs.extend(jobs_link)
    print(f"Fetched job links from {len(jobs_link)} jobs at {company_url} Successfully.")
    time.sleep(random.uniform(1, 3))
print(f'Total job links fetched: {len(all_jobs)}')

Fetched job links from 25 jobs at https://www.dice.com/company-profile/5bea0a06-736c-43a2-8da0-ef013448b47e?companyname=Navy%20Federal%20Credit%20Union Successfully.
Fetched job links from 5 jobs at https://www.dice.com/company-profile/f204eb07-4f05-493a-8f4f-a7aa98e9d5fc?companyname=Palmer%20Group Successfully.
Fetched job links from 13 jobs at https://www.dice.com/company-profile/18681f94-38b1-52d9-a5b0-6bd2ec1b0505?companyname=Luxoft%20USA%20Inc Successfully.
Fetched job links from 12 jobs at https://www.dice.com/company-profile/9fd6447b-0f42-5feb-a43e-3c9ce50c7526?companyname=HireTeq Successfully.
Fetched job links from 0 jobs at https://www.dice.com/company-profile/422f1aaf-adfd-4912-9b8b-1b86b4342681?companyname=Atlas%20Copco%20Group Successfully.
Fetched job links from 7 jobs at https://www.dice.com/company-profile/6d4d399d-1146-4cbf-9597-0096478fda35?companyname=Ornate%20Tech Successfully.
Fetched job links from 4 jobs at https://www.dice.com/company-profile/949ae65d-4208-5f57-

In [105]:
import pickle

with open('Jobs_links.pkl', 'wb') as f:
  pickle.dump(all_jobs, f)
print("Saved: Jobs_links.pkl")

Saved: Jobs_links.pkl


In [3]:
import pickle
with open('Jobs_links.pkl', 'rb') as f:
  loaded_config = pickle.load(f)
print("Loaded Jobs_links.pkl successfully.")
print(f"แสดงข้อมูลที่อ่านได้: {len(loaded_config)} บริษัท")
print(f"อ่านข้อมูลจาก Jobs_links.pkl: {loaded_config}")
print(f"เช็ค Data Type: {type(loaded_config)}")

Loaded Jobs_links.pkl successfully.
แสดงข้อมูลที่อ่านได้: 2414 บริษัท
อ่านข้อมูลจาก Jobs_links.pkl: ['https://www.dice.com/job-detail/e33e86a4-e3d9-46f8-b25d-83f9c4394cfa', 'https://www.dice.com/job-detail/53ad820a-7600-4efc-b1a9-b107de45775c', 'https://www.dice.com/job-detail/4d52f1b4-8109-453b-a5b5-2dbcdbf24737', 'https://www.dice.com/job-detail/8c760814-f872-470b-a849-bd21d8882015', 'https://www.dice.com/job-detail/18fad051-76da-414d-b485-454d6be7b5e4', 'https://www.dice.com/job-detail/3a226f4e-e149-4216-bc6c-b069ebd7333e', 'https://www.dice.com/job-detail/9ce029d6-fd43-4231-8cf3-40ac4d3f0de1', 'https://www.dice.com/job-detail/8972aa9d-d03a-4173-ae5f-4e73c299977c', 'https://www.dice.com/job-detail/71286a50-96cf-420c-866c-9bf5dfdac4a7', 'https://www.dice.com/job-detail/3cdcea52-206d-4604-9cd2-45ba85010fe7', 'https://www.dice.com/job-detail/12401c3a-6473-4bfc-a852-40a86269a58c', 'https://www.dice.com/job-detail/f033b887-447c-480f-b77e-d52c86e6a3e0', 'https://www.dice.com/job-detail/bf

Jobs Details

In [64]:
url = loaded_config[0]
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
company_name = soup.find('a', attrs={'data-wa-click': 'djv-job-company-profile-click'})
skills = soup.find_all('li', class_='text-sm text-gray-700')
job_name = soup.find('h1', class_='text-pretty font-bold text-2xl sm:text-2xl md:text-2xl lg:text-3xl xl:text-3xl 2xl:text-3xl')
print(f'company_name = {company_name.text.strip()}')
print(f'job_name = {job_name.text.strip()}')
print('skills = ')
for skill in skills:    
    print(f'- {skill.text.strip()}')

company_name = Navy Federal Credit Union
job_name = Principal Risk Analyst  (Reporting and Metrics)
skills = 
- Continuous Improvement
- FOCUS
- Knowledge Base
- Collaboration
- Testing
- Regulatory Compliance
- Quality Control
- Policies and Procedures
- Risk Assessment
- Mentorship
- Risk Management
- Process Improvement
- Research
- Analytical Skill
- Business Administration
- Auditing
- Banking
- Finance
- Communication
- Training
- Management
- Reporting
- Data Analysis
- Visualization
- Microsoft Power BI
- Tableau
- SQL
- Microsoft Excel
- Python
- Microsoft PowerPoint
- ServiceNow
- Performance Management
- Preventive Maintenance
- Project Management
- Military
- Media
- IP
- Intellectual Property
- SAP BASIS
- Law
- Recruiting


In [4]:
def fetch_job_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    dice_id = soup.find('li', attrs={'data-testid': 'legalInfo-companyName'})
    company_name = soup.find('a', attrs={'data-wa-click': 'djv-job-company-profile-click'})
    skills = soup.find_all('li', class_='text-sm text-gray-700')
    job_name = soup.find('h1', class_='text-pretty font-bold text-2xl sm:text-2xl md:text-2xl lg:text-3xl xl:text-3xl 2xl:text-3xl')
    posted_element = soup.find_all('div', {"class": 'mt-4'})
    start_li = soup.find('li', {'data-testid': 'legalInfo-companyName'})

    if start_li:
        # find_next_siblings จะหา li ตัวอื่นๆ ที่อยู่ในระดับเดียวกัน
        siblings = start_li.find_next_siblings('li')
        
        for s in siblings:
            if 'Posted' in s.get_text():
                result = s.get_text(strip=True).replace('Posted', '').strip()
                break
        else:
            result = None
    
    return {
        "post_id": dice_id.text.split()[2] if dice_id else None,
        "company_name": company_name.get_text(strip=True) if company_name else None,
        "job_name": job_name.get_text(strip=True) if job_name else None,
        "skills": [skill.get_text(strip=True) for skill in skills] if skills else [],
        "date_scraped": time.strftime("%Y-%m-%d"),
        "date_posted": result
    }


In [5]:
jobs_details_list = []

for i, job_url in enumerate(loaded_config[:500]):  # Fetch details for first 500 jobs
    try:
        print(f"Fetching details for URL: {job_url}")
        job_details = fetch_job_details(job_url)
        jobs_details_list.append(job_details)
        print(f'{i+1}. fetched successfully.')
        time.sleep(random.uniform(1, 3))
    except Exception as e:
        print(f'Error fetching job details from {job_url}: {e}')
    finally:
        continue
print(f'Total job details fetched: {len(jobs_details_list)}')

Fetching details for URL: https://www.dice.com/job-detail/e33e86a4-e3d9-46f8-b25d-83f9c4394cfa
1. fetched successfully.
Fetching details for URL: https://www.dice.com/job-detail/53ad820a-7600-4efc-b1a9-b107de45775c
2. fetched successfully.
Fetching details for URL: https://www.dice.com/job-detail/4d52f1b4-8109-453b-a5b5-2dbcdbf24737
3. fetched successfully.
Fetching details for URL: https://www.dice.com/job-detail/8c760814-f872-470b-a849-bd21d8882015
4. fetched successfully.
Fetching details for URL: https://www.dice.com/job-detail/18fad051-76da-414d-b485-454d6be7b5e4
5. fetched successfully.
Fetching details for URL: https://www.dice.com/job-detail/3a226f4e-e149-4216-bc6c-b069ebd7333e
6. fetched successfully.
Fetching details for URL: https://www.dice.com/job-detail/9ce029d6-fd43-4231-8cf3-40ac4d3f0de1
7. fetched successfully.
Fetching details for URL: https://www.dice.com/job-detail/8972aa9d-d03a-4173-ae5f-4e73c299977c
8. fetched successfully.
Fetching details for URL: https://www.di

In [6]:
import json
with open('jobs_details.json', 'w') as f:
    json.dump(jobs_details_list, f, indent=4)
print("Saved: jobs_details.json")

Saved: jobs_details.json


In [62]:
jobs_details_list

[{'post_id': '10117743',
  'company_name': 'Navy Federal Credit Union',
  'job_name': 'Principal Risk Analyst  (Reporting and Metrics)',
  'skills': ['Continuous Improvement',
   'FOCUS',
   'Knowledge Base',
   'Collaboration',
   'Testing',
   'Regulatory Compliance',
   'Quality Control',
   'Policies and Procedures',
   'Risk Assessment',
   'Mentorship',
   'Risk Management',
   'Process Improvement',
   'Research',
   'Analytical Skill',
   'Business Administration',
   'Auditing',
   'Banking',
   'Finance',
   'Communication',
   'Training',
   'Management',
   'Reporting',
   'Data Analysis',
   'Visualization',
   'Microsoft Power BI',
   'Tableau',
   'SQL',
   'Microsoft Excel',
   'Python',
   'Microsoft PowerPoint',
   'ServiceNow',
   'Performance Management',
   'Preventive Maintenance',
   'Project Management',
   'Military',
   'Media',
   'IP',
   'Intellectual Property',
   'SAP BASIS',
   'Law',
   'Recruiting'],
  'date_scraped': '2026-02-08',
  'date_posted': '1 